In [1]:
import numpy as np 
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
import pickle
import sklearn
print(sklearn.__version__)

1.0.2


# Importing Data

In [2]:
match=pd.read_csv("D:\Programs\Python\ML_notebooks\IPL_prediction\IPL_Matches_2008_2022.csv")
ball=pd.read_csv("D:\Programs\Python\ML_notebooks\IPL_prediction\IPL_Ball_by_Ball_2008_2022.csv")

In [3]:
ball.head()

,ID,innings,overs,ballnumber,batter,bowler,non-striker,extra_type,batsman_run,extras_run,total_run,non_boundary,isWicketDelivery,player_out,kind,fielders_involved,BattingTeam
0,1312200,1,0,1,YBK Jaiswal,Mohammed Shami,JC Buttler,NaN,0,0,0,0,0,NaN,NaN,NaN,Rajasthan Royals
1,1312200,1,0,2,YBK Jaiswal,Mohammed Shami,JC Buttler,legbyes,0,1,1,0,0,NaN,NaN,NaN,Rajasthan Royals
2,1312200,1,0,3,JC Buttler,Mohammed Shami,YBK Jaiswal,NaN,1,0,1,0,0,NaN,NaN,NaN,Rajasthan Royals
3,1312200,1,0,4,YBK Jaiswal,Mohammed Shami,JC Buttler,NaN,0,0,0,0,0,NaN,NaN,NaN,Rajasthan Royals
4,1312200,1,0,5,YBK Jaiswal,Mohammed Shami,JC Buttler,NaN,0,0,0,0,0,NaN,NaN,NaN,Rajasthan Royals


# Data modification

In [4]:
total_score_df=ball.groupby(['ID','innings']).sum()['total_run'].reset_index()
total_score_df=total_score_df[total_score_df['innings']==1]


In [5]:
match_df=match.merge(total_score_df[['ID','total_run']],left_on='ID',right_on='ID')

In [6]:
match_df

,ID,City,Date,Season,MatchNumber,Team1,Team2,Venue,TossWinner,TossDecision,...,WinningTeam,WonBy,Margin,method,Player_of_Match,Team1Players,Team2Players,Umpire1,Umpire2,total_run
0,1312200,Ahmedabad,2022-05-29,2022,Final,Rajasthan Royals,Gujarat Titans,"Narendra Modi Stadium, Ahmedabad",Rajasthan Royals,bat,...,Gujarat Titans,Wickets,7.0,NaN,HH Pandya,"['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...","['WP Saha', 'Shubman Gill', 'MS Wade', 'HH Pan...",CB Gaffaney,Nitin Menon,130
1,1312199,Ahmedabad,2022-05-27,2022,Qualifier 2,Royal Challengers Bangalore,Rajasthan Royals,"Narendra Modi Stadium, Ahmedabad",Rajasthan Royals,field,...,Rajasthan Royals,Wickets,7.0,NaN,JC Buttler,"['V Kohli', 'F du Plessis', 'RM Patidar', 'GJ ...","['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...",CB Gaffaney,Nitin Menon,157
2,1312198,Kolkata,2022-05-25,2022,Eliminator,Royal Challengers Bangalore,Lucknow Super Giants,"Eden Gardens, Kolkata",Lucknow Super Giants,field,...,Royal Challengers Bangalore,Runs,14.0,NaN,RM Patidar,"['V Kohli', 'F du Plessis', 'RM Patidar', 'GJ ...","['Q de Kock', 'KL Rahul', 'M Vohra', 'DJ Hooda...",J Madanagopal,MA Gough,207
3,1312197,Kolkata,2022-05-24,2022,Qualifier 1,Rajasthan Royals,Gujarat Titans,"Eden Gardens, Kolkata",Gujarat Titans,field,...,Gujarat Titans,Wickets,7.0,NaN,DA Miller,"['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...","['WP Saha', 'Shubman Gill', 'MS Wade', 'HH Pan...",BNJ Oxenford,VK Sharma,188
4,1304116,Mumbai,2022-05-22,2022,70,Sunrisers Hyderabad,Punjab Kings,"Wankhede Stadium, Mumbai",Sunrisers Hyderabad,bat,...,Punjab Kings,Wickets,5.0,NaN,Harpreet Brar,"['PK Garg', 'Abhishek Sharma', 'RA Tripathi', ...","['JM Bairstow', 'S Dhawan', 'M Shahrukh Khan',...",AK Chaudhary,NA Patwardhan,157
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
945,335986,Kolkata,2008-04-20,2007/08,4,Kolkata Knight Riders,Deccan Chargers,Eden Gardens,Deccan Chargers,bat,...,Kolkata Knight Riders,Wickets,5.0,NaN,DJ Hussey,"['WP Saha', 'BB McCullum', 'RT Ponting', 'SC G...","['AC Gilchrist', 'Y Venugopal Rao', 'VVS Laxma...",BF Bowden,K Hariharan,110
946,335985,Mumbai,2008-04-20,2007/08,5,Mumbai Indians,Royal Challengers Bangalore,Wankhede Stadium,Mumbai Indians,bat,...,Royal Challengers Bangalore,Wickets,5.0,NaN,MV Boucher,"['L Ronchi', 'ST Jayasuriya', 'DJ Thornely', '...","['S Chanderpaul', 'R Dravid', 'LRPL Taylor', '...",SJ Davis,DJ Harper,165
947,335984,Delhi,2008-04-19,2007/08,3,Delhi Daredevils,Rajasthan Royals,Feroz Shah Kotla,Rajasthan Royals,bat,...,Delhi Daredevils,Wickets,9.0,NaN,MF Maharoof,"['G Gambhir', 'V Sehwag', 'S Dhawan', 'MK Tiwa...","['T Kohli', 'YK Pathan', 'SR Watson', 'M Kaif'...",Aleem Dar,GA Pratapkumar,129
948,335983,Chandigarh,2008-04-19,2007/08,2,Kings XI Punjab,Chennai Super Kings,"Punjab Cricket Association Stadium, Mohali",Chennai Super Kings,bat,...,Chennai Super Kings,Runs,33.0,NaN,MEK Hussey,"['K Goel', 'JR Hopes', 'KC Sangakkara', 'Yuvra...","['PA Patel', 'ML Hayden', 'MEK Hussey', 'MS Dh...",MR Benson,SL Shastri,240


In [7]:
match_df['Team1'].unique()

array(['Rajasthan Royals', 'Royal Challengers Bangalore',
       'Sunrisers Hyderabad', 'Delhi Capitals', 'Chennai Super Kings',
       'Gujarat Titans', 'Lucknow Super Giants', 'Kolkata Knight Riders',
       'Punjab Kings', 'Mumbai Indians', 'Kings XI Punjab',
       'Delhi Daredevils', 'Rising Pune Supergiant', 'Gujarat Lions',
       'Rising Pune Supergiants', 'Pune Warriors', 'Deccan Chargers',
       'Kochi Tuskers Kerala'], dtype=object)

In [8]:
teams=['Rajasthan Royals', 'Royal Challengers Bangalore',
'Sunrisers Hyderabad', 'Delhi Capitals', 'Chennai Super Kings',
'Gujarat Titans', 'Lucknow Super Giants', 'Kolkata Knight Riders',
'Punjab Kings', 'Mumbai Indians',]

In [9]:
match_df['Team1']=match_df['Team1'].str.replace('Delhi Daredevils','Delhi Capitals')
match_df['Team2']=match_df['Team2'].str.replace('Delhi Daredevils','Delhi Capitals')

match_df['Team1']=match_df['Team1'].str.replace('Deccan Chargers','Sunrisers Hyderabad')
match_df['Team2']=match_df['Team2'].str.replace('Deccan Chargers','Sunrisers Hyderabad')

In [10]:
match_df=match_df[match_df['Team1'].isin(teams)]
match_df=match_df[match_df['Team2'].isin(teams)]

In [11]:
match_df

,ID,City,Date,Season,MatchNumber,Team1,Team2,Venue,TossWinner,TossDecision,...,WinningTeam,WonBy,Margin,method,Player_of_Match,Team1Players,Team2Players,Umpire1,Umpire2,total_run
0,1312200,Ahmedabad,2022-05-29,2022,Final,Rajasthan Royals,Gujarat Titans,"Narendra Modi Stadium, Ahmedabad",Rajasthan Royals,bat,...,Gujarat Titans,Wickets,7.0,NaN,HH Pandya,"['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...","['WP Saha', 'Shubman Gill', 'MS Wade', 'HH Pan...",CB Gaffaney,Nitin Menon,130
1,1312199,Ahmedabad,2022-05-27,2022,Qualifier 2,Royal Challengers Bangalore,Rajasthan Royals,"Narendra Modi Stadium, Ahmedabad",Rajasthan Royals,field,...,Rajasthan Royals,Wickets,7.0,NaN,JC Buttler,"['V Kohli', 'F du Plessis', 'RM Patidar', 'GJ ...","['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...",CB Gaffaney,Nitin Menon,157
2,1312198,Kolkata,2022-05-25,2022,Eliminator,Royal Challengers Bangalore,Lucknow Super Giants,"Eden Gardens, Kolkata",Lucknow Super Giants,field,...,Royal Challengers Bangalore,Runs,14.0,NaN,RM Patidar,"['V Kohli', 'F du Plessis', 'RM Patidar', 'GJ ...","['Q de Kock', 'KL Rahul', 'M Vohra', 'DJ Hooda...",J Madanagopal,MA Gough,207
3,1312197,Kolkata,2022-05-24,2022,Qualifier 1,Rajasthan Royals,Gujarat Titans,"Eden Gardens, Kolkata",Gujarat Titans,field,...,Gujarat Titans,Wickets,7.0,NaN,DA Miller,"['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...","['WP Saha', 'Shubman Gill', 'MS Wade', 'HH Pan...",BNJ Oxenford,VK Sharma,188
4,1304116,Mumbai,2022-05-22,2022,70,Sunrisers Hyderabad,Punjab Kings,"Wankhede Stadium, Mumbai",Sunrisers Hyderabad,bat,...,Punjab Kings,Wickets,5.0,NaN,Harpreet Brar,"['PK Garg', 'Abhishek Sharma', 'RA Tripathi', ...","['JM Bairstow', 'S Dhawan', 'M Shahrukh Khan',...",AK Chaudhary,NA Patwardhan,157
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
943,335988,Hyderabad,2008-04-22,2007/08,7,Sunrisers Hyderabad,Delhi Capitals,"Rajiv Gandhi International Stadium, Uppal",Deccan Chargers,bat,...,Delhi Daredevils,Wickets,9.0,NaN,V Sehwag,"['AC Gilchrist', 'Y Venugopal Rao', 'VVS Laxma...","['G Gambhir', 'V Sehwag', 'S Dhawan', 'Shoaib ...",IL Howell,AM Saheba,142
945,335986,Kolkata,2008-04-20,2007/08,4,Kolkata Knight Riders,Sunrisers Hyderabad,Eden Gardens,Deccan Chargers,bat,...,Kolkata Knight Riders,Wickets,5.0,NaN,DJ Hussey,"['WP Saha', 'BB McCullum', 'RT Ponting', 'SC G...","['AC Gilchrist', 'Y Venugopal Rao', 'VVS Laxma...",BF Bowden,K Hariharan,110
946,335985,Mumbai,2008-04-20,2007/08,5,Mumbai Indians,Royal Challengers Bangalore,Wankhede Stadium,Mumbai Indians,bat,...,Royal Challengers Bangalore,Wickets,5.0,NaN,MV Boucher,"['L Ronchi', 'ST Jayasuriya', 'DJ Thornely', '...","['S Chanderpaul', 'R Dravid', 'LRPL Taylor', '...",SJ Davis,DJ Harper,165
947,335984,Delhi,2008-04-19,2007/08,3,Delhi Capitals,Rajasthan Royals,Feroz Shah Kotla,Rajasthan Royals,bat,...,Delhi Daredevils,Wickets,9.0,NaN,MF Maharoof,"['G Gambhir', 'V Sehwag', 'S Dhawan', 'MK Tiwa...","['T Kohli', 'YK Pathan', 'SR Watson', 'M Kaif'...",Aleem Dar,GA Pratapkumar,129


In [12]:
match_df=match_df[match_df['method'].isna()]

In [13]:
match_df=match_df[['ID','City','WinningTeam','total_run','Team1','Team2']]

In [14]:
ball_df=match_df.merge(ball,on='ID')
ball_df=ball_df[ball_df['innings']==2]
ball_df

,ID,City,WinningTeam,total_run_x,Team1,Team2,innings,overs,ballnumber,batter,...,extra_type,batsman_run,extras_run,total_run_y,non_boundary,isWicketDelivery,player_out,kind,fielders_involved,BattingTeam
120,1312200,Ahmedabad,Gujarat Titans,130,Rajasthan Royals,Gujarat Titans,2,0,1,WP Saha,...,NaN,0,0,0,0,0,NaN,NaN,NaN,Gujarat Titans
121,1312200,Ahmedabad,Gujarat Titans,130,Rajasthan Royals,Gujarat Titans,2,0,2,WP Saha,...,NaN,0,0,0,0,0,NaN,NaN,NaN,Gujarat Titans
122,1312200,Ahmedabad,Gujarat Titans,130,Rajasthan Royals,Gujarat Titans,2,0,3,WP Saha,...,NaN,1,0,1,0,0,NaN,NaN,NaN,Gujarat Titans
123,1312200,Ahmedabad,Gujarat Titans,130,Rajasthan Royals,Gujarat Titans,2,0,4,Shubman Gill,...,NaN,0,0,0,0,0,NaN,NaN,NaN,Gujarat Titans
124,1312200,Ahmedabad,Gujarat Titans,130,Rajasthan Royals,Gujarat Titans,2,0,5,Shubman Gill,...,NaN,0,0,0,0,0,NaN,NaN,NaN,Gujarat Titans
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155811,335982,Bangalore,Kolkata Knight Riders,222,Royal Challengers Bangalore,Kolkata Knight Riders,2,14,5,P Kumar,...,legbyes,0,1,1,0,0,NaN,NaN,NaN,Royal Challengers Bangalore
155812,335982,Bangalore,Kolkata Knight Riders,222,Royal Challengers Bangalore,Kolkata Knight Riders,2,14,6,SB Joshi,...,NaN,1,0,1,0,0,NaN,NaN,NaN,Royal Challengers Bangalore
155813,335982,Bangalore,Kolkata Knight Riders,222,Royal Challengers Bangalore,Kolkata Knight Riders,2,14,7,P Kumar,...,NaN,0,0,0,0,0,NaN,NaN,NaN,Royal Challengers Bangalore
155814,335982,Bangalore,Kolkata Knight Riders,222,Royal Challengers Bangalore,Kolkata Knight Riders,2,15,1,SB Joshi,...,wides,0,1,1,0,0,NaN,NaN,NaN,Royal Challengers Bangalore


In [15]:
ball_df["Current_Score"]=ball_df.groupby('ID').cumsum()['total_run_y']
ball_df["Runs_left"]=ball_df['total_run_x']-ball_df['Current_Score']+1

In [16]:
ball_df['Balls_left']=120-(ball_df['overs']*6+ball_df['ballnumber'])
ball_df

,ID,City,WinningTeam,total_run_x,Team1,Team2,innings,overs,ballnumber,batter,...,total_run_y,non_boundary,isWicketDelivery,player_out,kind,fielders_involved,BattingTeam,Current_Score,Runs_left,Balls_left
120,1312200,Ahmedabad,Gujarat Titans,130,Rajasthan Royals,Gujarat Titans,2,0,1,WP Saha,...,0,0,0,NaN,NaN,NaN,Gujarat Titans,0,131,119
121,1312200,Ahmedabad,Gujarat Titans,130,Rajasthan Royals,Gujarat Titans,2,0,2,WP Saha,...,0,0,0,NaN,NaN,NaN,Gujarat Titans,0,131,118
122,1312200,Ahmedabad,Gujarat Titans,130,Rajasthan Royals,Gujarat Titans,2,0,3,WP Saha,...,1,0,0,NaN,NaN,NaN,Gujarat Titans,1,130,117
123,1312200,Ahmedabad,Gujarat Titans,130,Rajasthan Royals,Gujarat Titans,2,0,4,Shubman Gill,...,0,0,0,NaN,NaN,NaN,Gujarat Titans,1,130,116
124,1312200,Ahmedabad,Gujarat Titans,130,Rajasthan Royals,Gujarat Titans,2,0,5,Shubman Gill,...,0,0,0,NaN,NaN,NaN,Gujarat Titans,1,130,115
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155811,335982,Bangalore,Kolkata Knight Riders,222,Royal Challengers Bangalore,Kolkata Knight Riders,2,14,5,P Kumar,...,1,0,0,NaN,NaN,NaN,Royal Challengers Bangalore,80,143,31
155812,335982,Bangalore,Kolkata Knight Riders,222,Royal Challengers Bangalore,Kolkata Knight Riders,2,14,6,SB Joshi,...,1,0,0,NaN,NaN,NaN,Royal Challengers Bangalore,81,142,30
155813,335982,Bangalore,Kolkata Knight Riders,222,Royal Challengers Bangalore,Kolkata Knight Riders,2,14,7,P Kumar,...,0,0,0,NaN,NaN,NaN,Royal Challengers Bangalore,81,142,29
155814,335982,Bangalore,Kolkata Knight Riders,222,Royal Challengers Bangalore,Kolkata Knight Riders,2,15,1,SB Joshi,...,1,0,0,NaN,NaN,NaN,Royal Challengers Bangalore,82,141,29


In [17]:
ball_df["Wickets_left"]=10-ball_df.groupby('ID').cumsum()['isWicketDelivery']

In [18]:
ball_df["CRR"]=(ball_df['Current_Score']*6)/(120-ball_df['Balls_left'])
ball_df["RRR"]=(ball_df["Runs_left"]*6)/ball_df['Balls_left']

In [19]:
def result(row):
    if row['BattingTeam']==row['WinningTeam']:
        return 1
    else:
        return 0

In [20]:
ball_df['Result']=ball_df.apply(result,axis=1)

In [22]:
def BT(row):
    if row['BattingTeam']==row['Team1']:
        return row['Team2']
    else:
        return row['Team1']

In [23]:
ball_df['BowlingTeam']=ball_df.apply(BT,axis=1)
ball_df

,ID,City,WinningTeam,total_run_x,Team1,Team2,innings,overs,ballnumber,batter,...,fielders_involved,BattingTeam,Current_Score,Runs_left,Balls_left,Wickets_left,CRR,RRR,Result,BowlingTeam
120,1312200,Ahmedabad,Gujarat Titans,130,Rajasthan Royals,Gujarat Titans,2,0,1,WP Saha,...,NaN,Gujarat Titans,0,131,119,10,0.000000,6.605042,1,Rajasthan Royals
121,1312200,Ahmedabad,Gujarat Titans,130,Rajasthan Royals,Gujarat Titans,2,0,2,WP Saha,...,NaN,Gujarat Titans,0,131,118,10,0.000000,6.661017,1,Rajasthan Royals
122,1312200,Ahmedabad,Gujarat Titans,130,Rajasthan Royals,Gujarat Titans,2,0,3,WP Saha,...,NaN,Gujarat Titans,1,130,117,10,2.000000,6.666667,1,Rajasthan Royals
123,1312200,Ahmedabad,Gujarat Titans,130,Rajasthan Royals,Gujarat Titans,2,0,4,Shubman Gill,...,NaN,Gujarat Titans,1,130,116,10,1.500000,6.724138,1,Rajasthan Royals
124,1312200,Ahmedabad,Gujarat Titans,130,Rajasthan Royals,Gujarat Titans,2,0,5,Shubman Gill,...,NaN,Gujarat Titans,1,130,115,10,1.200000,6.782609,1,Rajasthan Royals
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155811,335982,Bangalore,Kolkata Knight Riders,222,Royal Challengers Bangalore,Kolkata Knight Riders,2,14,5,P Kumar,...,NaN,Royal Challengers Bangalore,80,143,31,1,5.393258,27.677419,0,Kolkata Knight Riders
155812,335982,Bangalore,Kolkata Knight Riders,222,Royal Challengers Bangalore,Kolkata Knight Riders,2,14,6,SB Joshi,...,NaN,Royal Challengers Bangalore,81,142,30,1,5.400000,28.400000,0,Kolkata Knight Riders
155813,335982,Bangalore,Kolkata Knight Riders,222,Royal Challengers Bangalore,Kolkata Knight Riders,2,14,7,P Kumar,...,NaN,Royal Challengers Bangalore,81,142,29,1,5.340659,29.379310,0,Kolkata Knight Riders
155814,335982,Bangalore,Kolkata Knight Riders,222,Royal Challengers Bangalore,Kolkata Knight Riders,2,15,1,SB Joshi,...,NaN,Royal Challengers Bangalore,82,141,29,1,5.406593,29.172414,0,Kolkata Knight Riders


In [80]:
final_df=ball_df[['BattingTeam','BowlingTeam','City','Runs_left','Balls_left','Wickets_left','total_run_x','CRR','RRR','Result']]
final_df=final_df.sample(final_df.shape[0])
final_df.dropna(inplace=True)
final_df=final_df[final_df['Balls_left']!=0]
final_df['City']=final_df['City'].str.replace('Bangalore','Bengaluru')
final_df

,BattingTeam,BowlingTeam,City,Runs_left,Balls_left,Wickets_left,total_run_x,CRR,RRR,Result
49486,Sunrisers Hyderabad,Delhi Capitals,Hyderabad,72,43,8,155,6.545455,10.046512,0
73328,Mumbai Indians,Delhi Capitals,Delhi,63,46,8,164,8.270270,8.217391,0
54262,Chennai Super Kings,Royal Challengers Bangalore,Chennai,58,92,9,70,2.785714,3.782609,1
56246,Chennai Super Kings,Delhi Capitals,Delhi,93,61,9,162,7.118644,9.147541,0
134536,Delhi Daredevils,Rajasthan Royals,Ahmedabad,128,112,10,141,10.500000,6.857143,1
...,...,...,...,...,...,...,...,...,...,...
155540,Delhi Daredevils,Delhi Capitals,Delhi,65,75,9,129,8.666667,5.200000,1
107033,Delhi Daredevils,Delhi Capitals,Pune,80,56,7,162,7.781250,8.571429,0
21703,Punjab Kings,Kolkata Knight Riders,Dubai,44,28,8,165,7.956522,9.428571,1
33697,Delhi Capitals,Royal Challengers Bangalore,Abu Dhabi,106,88,9,152,8.812500,7.227273,1


In [81]:
X=final_df.iloc[:,:-1]
Y=final_df.iloc[:,-1]
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=1)

In [82]:
trf=ColumnTransformer([('trf', OneHotEncoder(sparse=False,drop='first'),['BattingTeam','BowlingTeam','City'])]
                        ,remainder='passthrough')

# Training Model

In [83]:
pipe=Pipeline(steps=[
        ('step1',trf),
        ('step2',LogisticRegression(solver='liblinear'))
])

In [84]:
pipe.fit(X_train,Y_train)

Pipeline(steps=[('step1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('trf',
                                                  OneHotEncoder(drop='first',
                                                                sparse=False),
                                                  ['BattingTeam', 'BowlingTeam',
                                                   'City'])])),
                ('step2', LogisticRegression(solver='liblinear'))])

In [85]:
y_pred=pipe.predict(X_test)
accuracy_score(Y_test,y_pred)

0.8289212910532276

In [86]:
final_df['City'].unique()

array(['Hyderabad', 'Delhi', 'Chennai', 'Ahmedabad', 'Jaipur', 'Kolkata',
       'Mumbai', 'Dubai', 'Bengaluru', 'Kimberley', 'Abu Dhabi', 'Pune',
       'Johannesburg', 'Port Elizabeth', 'Navi Mumbai', 'Visakhapatnam',
       'Ranchi', 'Durban', 'Sharjah', 'Cape Town', 'Centurion', 'Nagpur',
       'Cuttack', 'Raipur', 'East London', 'Bloemfontein'], dtype=object)

In [87]:
pickle.dump(pipe,open('pipe.pkl','wb'))

Prediction will be done through a website